# Neural Network Policy for the Occluded Crosswalk Problem

In [1]:
using POMDPs, StatsBase, POMDPToolbox, SARSOP, QMDP, PyCall
using AutomotiveDrivingModels, AutoViz, Reel, Images, JLD, Parameters

In [2]:
include("../../admenv/crosswalk/occluded_crosswalk_env.jl")
include("../../admenv/crosswalk/pomdp_types.jl")
include("../../admenv/crosswalk/spaces.jl")
include("../../admenv/crosswalk/transition.jl")
include("../../admenv/crosswalk/observation.jl")
include("../../admenv/crosswalk/belief.jl")
include("../../admenv/crosswalk/adm_helpers.jl")
include("../../admenv/crosswalk/decomposition.jl")
include("../../admenv/crosswalk/render_helpers.jl")

In [3]:
include("config.jl")
include("ego_control.jl")
include("sensor.jl")
include("policy.jl")
include("state_estimation.jl")
include("constant_pedestrian.jl")
include("simulation.jl")
include("baseline_policy.jl")
include("nn_eval_model.jl")
include("render_helpers.jl")

## Initialization

In [18]:
config = EvalConfig(time_out = 200, n_ped=2, n_episodes = 10)

EvalConfig 
	 Sim step (s) 0.10 
	 N episodes   10 
	 Time out  200 
	 Time out  200 


In [5]:
cam = FitToContentCamera(0.0)

AutoViz.FitToContentCamera(0.0)

In [6]:
params = EnvParams(ped_rate = 0.03)
env = CrosswalkEnv(params);

In [7]:
pomdp = OCPOMDP();

#### Import NN Policy

In [8]:
@pyimport tensorflow as tf
nn_wrapper = pyimport("dqn.nn_wrapper")

PyObject <module 'dqn.nn_wrapper' from 'C:\\Users\\Maxime\\OneDrive - Leland Stanford Junior University\\Research\\maximeworkspace\\dqn\\nn_wrapper.py'>

In [9]:
nn = nn_wrapper[:NNWrapper]("../../dqn/experiments\\test\\")

INFO:tensorflow:Restoring parameters from ../../dqn/experiments\test\model.ckpt


2017-10-02 15:19:23.431416: W c:\tf_jenkins\home\workspace\release-win\m\windows\py\35\tensorflow\core\platform\cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE instructions, but these are available on your machine and could speed up CPU computations.
2017-10-02 15:19:23.431443: W c:\tf_jenkins\home\workspace\release-win\m\windows\py\35\tensorflow\core\platform\cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE2 instructions, but these are available on your machine and could speed up CPU computations.
2017-10-02 15:19:23.431449: W c:\tf_jenkins\home\workspace\release-win\m\windows\py\35\tensorflow\core\platform\cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE3 instructions, but these are available on your machine and could speed up CPU computations.
2017-10-02 15:19:23.431454: W c:\tf_jenkins\home\workspace\release-win\m\windows\py\35\tensorflow\core\platform\cpu_feature_guard.cc:45] The TensorFlow library wasn

PyObject <dqn.nn_wrapper.NNWrapper object at 0x000000002A84ACC0>

In [10]:
policy = NNPolicy(pomdp, nn);

In [11]:
sensor = POMDPSensor(pomdp = pomdp, sensor = SimpleSensor(0.5, 0.5))

POMDPSensor
  pomdp: OCPOMDP
  sensor: SimpleSensor


In [12]:
up = NNUpdater(pomdp);
a0 = 0.
ego0 = initial_ego(env, config.rng)
b0 = initialize_off_belief(policy, ego0.state)
o0 = Dict{Int64, OCObs}()

Dict{Int64,OCState} with 0 entries

In [13]:
update_freq = 5
ego_model = CrosswalkDriver(update_freq, 0, env, a0, b0, o0, ego0, sensor, policy, up);

In [14]:
reset_model!(ego_model, ego0)

## Run a simulation 

In [15]:
ego0 = initial_ego(env, config.rng)
reset_model!(ego_model, ego0)
models = Dict{Int, DriverModel}()
models[1] = ego_model
scene = initial_scene(models, env, config)
push!(scene, ego0)
nticks = 200
rec = SceneRecord(nticks+1, config.sim_dt)

SceneRecord(nscenes=0)

In [16]:
# execute the simulation
simulate!(rec, scene, env, models, nticks, config.rng, config.n_ped, config.callbacks)

SceneRecord(nscenes=201)

In [17]:
duration, fps, render_rec = animate_record(rec)
speed_factor = 1
film = roll(render_rec, fps = fps*speed_factor, duration = duration/speed_factor)

Reel.Frames{MIME{Symbol("image/png")}}("C:\\Users\\Maxime\\AppData\\Local\\Temp\\jl_DC6E.tmp", 0x00000000000000c9, 10.0, nothing)

## Evaluation

In [19]:
crash = 0
avg_step = 0
success = 0
time_out = 0
avg_step = 0
for i=1:config.n_episodes    
    ego0 = initial_ego(env, config.rng)
    reset_model!(ego_model, ego0)
    models = Dict{Int, DriverModel}()
    models[1] = ego_model
    scene = initial_scene(models, env, config)
    push!(scene, ego0)
    nticks = 200
    rec = SceneRecord(nticks+1, config.sim_dt)
    # execute the simulation
    simulate!(rec, scene, env, models, nticks, config.rng, config.n_ped, config.callbacks)
    avg_step += nframes(rec)
    
    # analyse output 
    if is_crash(rec[0])
        crash += 1
        println("Episode $i CRASH")
#         break
    elseif is_terminal(rec[0], env)
        success += 1
        println("Episode $i SUCCESS")
    elseif nframes(rec) >= config.time_out
        time_out += 1
        println("Episode $i TIME OUT")
#         break
    else
        println("Episode $i nothing happened, frames: $(nframes(rec))")
    end
end
println("End of the simulation: Crashes $crash , Successes $success, Time out $time_out, Avg Step $(avg_step/config.n_episodes)")

Episode 1 TIME OUT
Episode 2 TIME OUT
Episode 3 TIME OUT
Episode 4 TIME OUT
Episode 5 TIME OUT
Episode 6 TIME OUT
Episode 7 TIME OUT
Episode 8 TIME OUT
Episode 9 TIME OUT
Episode 10 TIME OUT
End of the simulation: Crashes 0 , Successes 0, Time out 10, Avg Step 201.0
